## Generate Data

In [1]:
# dependences for data loading and sifting
import os
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [2]:
# file paths to data directories
train_dir = "data/train/"
test_dir = "data/test/"
val_dir = "data/val/"

In [3]:
# instatiating a data degenerater for each split sample 
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)
                                   
val_datagen = ImageDataGenerator(rescale=1./255)

train_data_generator = train_datagen.flow_from_directory(
                       train_dir,
                       target_size=(150,150),
                       batch_size=16,
                       class_mode='binary',
                       color_mode='grayscale')

test_data_generator = test_datagen.flow_from_directory(
                      test_dir,
                      target_size=(150,150),
                      batch_size=16,
                      class_mode='binary',
                      color_mode='grayscale')

val_data_generator = val_datagen.flow_from_directory(
                     val_dir,
                     target_size=(150,150),
                     batch_size=16,
                     class_mode='binary',
                     color_mode='grayscale')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


## Baseline Model
Beginning a convolutional neural network with the most simple architecture for the sake of efficient diagnostics and optimization.  

In [4]:
from keras import layers
from keras import models
from keras import optimizers

The architecture begins with a convolutional 2D layers, followed by a pooling layer, flattened, and then followed up with two dense layers.  
THe model is compiled using binary cross entropy to measure the loss function and accuracy foor the performance metric.

In [ ]:

# Your code here; design and compile the model
base_model = models.Sequential()
base_model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,1)))
base_model.add(layers.MaxPooling2D((2, 2)))
base_model.add(layers.Flatten())
base_model.add(layers.Dense(512, activation='relu'))
base_model.add(layers.Dense(1, activation='sigmoid'))


base_model.compile(loss='binary_crossentropy',
              metrics=['acc'])

the compiled base model is now fit very lean, using 10 steps per epoch and 5 total epochs, the test data set is used for validation as the actual validation sample is extremely small and reserved only for the final phase of performance evaluation. 

In [15]:
base_model_results = base_model.fit(train_data_generator, 
                              steps_per_epoch=10, 
                              epochs=5, 
                              validation_data=test_data_generator, 
                              validation_steps=15)

Epoch 1/5
10/10 [==============================] - 16s 2s/step - loss: 2.3194 - acc: 0.7625 - val_loss: 0.6574 - val_acc: 0.7333
Epoch 2/5
10/10 [==============================] - 13s 1s/step - loss: 0.4071 - acc: 0.8500 - val_loss: 0.4250 - val_acc: 0.8333
Epoch 3/5
10/10 [==============================] - 11s 1s/step - loss: 0.2486 - acc: 0.8938 - val_loss: 0.5820 - val_acc: 0.7458
Epoch 4/5
10/10 [==============================] - 11s 1s/step - loss: 0.3312 - acc: 0.8562 - val_loss: 0.8100 - val_acc: 0.7458
Epoch 5/5
10/10 [==============================] - 10s 1s/step - loss: 0.1977 - acc: 0.9250 - val_loss: 0.4881 - val_acc: 0.7958


The base model begins stronger than expected with an accuracy of around 70% on the test data (sometimes better or worse, re-running the cell multiple times). A good start, with plenty of room to improve. 

## Sophisticating model architecture

Before exploring tuning and optimization of hyperparameters and other methods such as regularization, we will sophisticate the architectuer of the model itself by incorporating more layers in a variety of types including convolutional layers, pooling layers, and dense layers. 

In [17]:
deep_model = models.Sequential()

deep_model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,1)))
deep_model.add(layers.MaxPooling2D((2, 2)))
deep_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
deep_model.add(layers.MaxPooling2D((2, 2)))
deep_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
deep_model.add(layers.MaxPooling2D((2, 2)))
deep_model.add(layers.Flatten())
deep_model.add(layers.Dense(512, activation='relu'))
deep_model.add(layers.Dense(256, activation='relu'))
deep_model.add(layers.Dense(1, activation='sigmoid'))

deep_model.compile(loss='binary_crossentropy',
              metrics=['acc'])

In [18]:
deep_model_results = deep_model.fit(train_data_generator, 
                              steps_per_epoch=10, 
                              epochs=5, 
                              validation_data=test_data_generator, 
                              validation_steps=15)

Epoch 1/5
10/10 [==============================] - 8s 819ms/step - loss: 2.0752 - acc: 0.6375 - val_loss: 0.7143 - val_acc: 0.6042
Epoch 2/5
10/10 [==============================] - 8s 825ms/step - loss: 0.5867 - acc: 0.7500 - val_loss: 1.2636 - val_acc: 0.6250
Epoch 3/5
10/10 [==============================] - 8s 802ms/step - loss: 0.6839 - acc: 0.7063 - val_loss: 0.4876 - val_acc: 0.8125
Epoch 4/5
10/10 [==============================] - 8s 755ms/step - loss: 0.9015 - acc: 0.7563 - val_loss: 0.5659 - val_acc: 0.6667
Epoch 5/5
10/10 [==============================] - 9s 912ms/step - loss: 0.4580 - acc: 0.8062 - val_loss: 0.5865 - val_acc: 0.6042
